In [17]:
import bitarray
import json
from hashlib import sha3_256, sha256, blake2b

class BloomFilter():

# with the following three hash functions and store the words in the bloom filter
    def my_hash(self, s):
        return int(sha256(s.lower().encode()).hexdigest(), 16) % self._size
    
    def my_hash2(self, s):
        return int(blake2b(s.lower().encode()).hexdigest(), 16) % self._size

    def my_hash3(self, s):
        return int(sha3_256(s.lower().encode()).hexdigest(), 16) % self._size   
    
    def initial(self, size=10_000_000, hash_dic=3):
        self._size = size
        self._arr = bitarray.bitarray(self._size)
        self._arr.setall(0)
        if hash_dic == 1:
            self._hash_dic = (self.my_hash,)
        elif hash_dic == 2:
            self._hash_dic = (self.my_hash, self.my_hash2)
        else:
            self._hash_dic = (self.my_hash, self.my_hash2, self.my_hash3)
            



# Implement a Bloom Filter "from scratch" using a bitarray               
    def __add__(self, x):
        for hash_ in self._hash_dic:
            self._arr[hash_(x)] = 1
    
    def __contains__(self, x):
        for hash_ in self._hash_dic:
            if (self._arr[hash_(x)] == 0):
                return False
        return True

# import word.txt file
        
class Dict_():
    
    def initial(self, size=10_000_000, hash_dic=3):
        self.BF = BloomFilter(size, hash_dic)
        with open('words.txt') as f:
            for line in f:
                word = line.strip()
                self.BF + word
                
    def __contains__(self, x):
        return x in self.BF
    
class SpellCheck():
    
    def initial(self, size=10_000_000, hash_dic=3):
        self._dict = Dict_(size, hash_dic)
    
    def possible_words(self, x):
        result = []
        x_split = list(x)
        for i in range(len(x_split)):
            temp = x_split[i]
            if x_split[i].isalpha():
                for j in range(97,123):
                    x_split[i] = chr(j)
                    if not "".join(x_split) in result:
                        result.append("".join(x_split))
            x_split[i] = temp
        return result   
    
    def suggestions(self, x):
        ls = self.possible_words(x)
        outls = []
        for item in ls:
            if item in self._dict:
                outls.append(item)
        return outls
        


In [19]:
sc = SpellCheck()
sc.possible_words('flower')

['alower',
 'blower',
 'clower',
 'dlower',
 'elower',
 'flower',
 'glower',
 'hlower',
 'ilower',
 'jlower',
 'klower',
 'llower',
 'mlower',
 'nlower',
 'olower',
 'plower',
 'qlower',
 'rlower',
 'slower',
 'tlower',
 'ulower',
 'vlower',
 'wlower',
 'xlower',
 'ylower',
 'zlower',
 'faower',
 'fbower',
 'fcower',
 'fdower',
 'feower',
 'ffower',
 'fgower',
 'fhower',
 'fiower',
 'fjower',
 'fkower',
 'fmower',
 'fnower',
 'foower',
 'fpower',
 'fqower',
 'frower',
 'fsower',
 'ftower',
 'fuower',
 'fvower',
 'fwower',
 'fxower',
 'fyower',
 'fzower',
 'flawer',
 'flbwer',
 'flcwer',
 'fldwer',
 'flewer',
 'flfwer',
 'flgwer',
 'flhwer',
 'fliwer',
 'fljwer',
 'flkwer',
 'fllwer',
 'flmwer',
 'flnwer',
 'flpwer',
 'flqwer',
 'flrwer',
 'flswer',
 'fltwer',
 'fluwer',
 'flvwer',
 'flwwer',
 'flxwer',
 'flywer',
 'flzwer',
 'floaer',
 'flober',
 'flocer',
 'floder',
 'floeer',
 'flofer',
 'floger',
 'floher',
 'floier',
 'flojer',
 'floker',
 'floler',
 'flomer',
 'floner',
 'flooer',